In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/Users/burak/Developer/MachineLearning/4462 - Final Project/notebooks/mock_user_data.csv')
df

,username,country,title,date_watched,percentage_watched,rating
0,'Bountiful Blessings' 'The OA' 'Mayday Life',NaN,NaN,NaN,NaN,NaN
1,"""""Ben & Holly's Little Kingdom"""" 'Super Nani']""",2008,25,8.0,NaN,NaN
2,Mark Lopez,Palestinian Territory,['Maniyarayile Ashokan' 'Good Game: The Beginn...,2010.0,67.0,6.0
3,Tammy Cooper,Slovenia,"['Doom: Annihilation' 'Klaus' ""Angela's Christ...",2013.0,70.0,4.0
4,Mckenzie George,Nigeria,['Killer Cove' 'Rocks' 'W. Kamau Bell: Private...,2000.0,54.0,7.0
...,...,...,...,...,...,...
14996,Alice Mathis,Canada,['El testigo' 'Trailer Park Boys: Say Goodnigh...,2020.0,41.0,2.0
14997,Melissa Bennett,Netherlands,['Chauthi Koot' 'Mystic Pop-up Bar' 'Never Hea...,2012.0,69.0,1.0
14998,James Mitchell,South Georgia and the South Sandwich Islands,['Entangled' 'Maharaja: The Story of Ranjit Si...,2003.0,2.0,10.0
14999,Tina Nguyen,French Guiana,['Lez Bomb' 'Beast Legends' 'Brown Nation' 'Je...,2003.0,13.0,7.0


In [6]:
#drop nan
df = df.dropna()

#drop username we don't need it for association learning
#df = df.drop(columns=['username'])

In [7]:
df.sample(3)

,username,country,title,date_watched,percentage_watched,rating
2547,Thomas Perez,Antigua and Barbuda,['Sad Hill Unearthed' 'Camarón: The Film'\n 'S...,2015.0,17.0,4.0
11983,Andrea Henderson,Moldova,['Made in Mexico' 'The Lincoln Lawyer' 'Pacifi...,2015.0,81.0,10.0
8621,Mark Moses,Japan,['Strongest Deliveryman' 'Alice Junior' 'Thera...,2006.0,6.0,7.0


In [8]:
df.dtypes

username               object
country                object
title                  object
date_watched          float64
percentage_watched    float64
rating                float64
dtype: object

In [9]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [15]:
def create_apriori_datastructure(df, id=True):
    """
    Function that creates the apriori datastructure.
    If there is an id, it will group by Invoice and StockCode.
    If there is no id, it will group by Invoice and Description.
    """
    if id:
        # Group by Invoice and StockCode.
        grouped = df.groupby(
            ['username', 'percentage_watched'],
            as_index=False
        ).agg({'rating': 'sum'})
        
        # Pivot the data.
        apriori_datastructure = pd.pivot(
            data=grouped,
            index='username',
            columns='percentage_watched',
            values='rating'
        ).fillna(0).applymap(lambda x: 1 if x > 0 else 0)
        
        return apriori_datastructure


df_apriori = create_apriori_datastructure(df)
print(df_apriori.dtypes)  

percentage_watched
0.0      int64
1.0      int64
2.0      int64
3.0      int64
4.0      int64
         ...  
96.0     int64
97.0     int64
98.0     int64
99.0     int64
100.0    int64
Length: 101, dtype: object


In [29]:
def get_rules(apriori_df):

    frequent_itemsets = apriori(
        apriori_df,
        min_support=0.01,
        use_colnames=True
    )

    print(frequent_itemsets.head())
    
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
    rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
    rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
    rules.sort_values("lift",ascending=False)

    #print(rules.head())
    
    return rules

rules = get_rules(df_apriori)

    support itemsets
0  0.011202    (1.0)
1  0.010909    (2.0)
2  0.010690    (3.0)
3  0.010031    (4.0)
4  0.010983    (5.0)


/Users/burak/miniforge3/envs/we-hear-you/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [26]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedents_length,consequents_length
